# Setup

Run this code to ensure you're working on the correct environment

In [1]:
import sys 
print(sys.executable)

C:\Users\nitin\miniconda3\envs\fyp_env\python.exe


Next let's import all the necessary libraries

In [2]:
# general
import os
import random
import numpy as np
import pandas as pd
from numpy.lib.npyio import save
import math

# visualization
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib

# for images handling & manipulation

from skimage.io import imread
from skimage import measure

from scipy.ndimage import gaussian_filter
from scipy.ndimage import laplace

from PIL import Image, ImageEnhance

Using matplotlib backend: Qt5Agg


### Declaring Constants

In [3]:
# setting the directory of our dataset
DATASET_DIR = "dataset/mias/all-mias/"

# Load Data

In [4]:
def get_img_paths(directory):
    """
    Stores the path to all the images in a directory.
    
    Parameters:
        directory(str): The directory containing the images
    
    Returns:
        list[str]: A list of paths to the pgm images
    """
    # to store the paths to all the images
    image_paths = []

    for filename in os.listdir(directory):
        # specifying image formats
        if filename.endswith(".pgm"):
            image_paths.append(os.path.join(directory, filename))
            
    return image_paths

In [5]:
def read_img_as_np_array(img_path, rgb=False):
    """
    Reads an image as a numpy array.
    
    Parameters:
        img_path(str): Path to the image
        rgb(bool): Whether the image should be converted to RGB. False by default.
    
    Returns:
        numpy.ndarray: Image as a numpy array
    """
    if rgb:
        return np.asarray(Image.open(img_path).convert('RGB'))
    else:
        return np.asarray(Image.open(img_path))

In [6]:
# store the image paths
image_paths = get_img_paths(DATASET_DIR)

# EDA

Performing some exploratory data analysis to understand the data at hand.

In [7]:
# output some basic stats
print('No. of images:', len(image_paths))

sample_img = read_img_as_np_array(image_paths[0])

print('Shape of image:', sample_img.shape)
print('Size of image:', sample_img.size, 'elements')

No. of images: 322
Shape of image: (1024, 1024)
Size of image: 1048576 elements


### Display the images

In [8]:
def display_image(img_info):
    """
    Displays an image from the dataset.
    
    Parameters:
        img_info(int|str): Either the number of the image or the path to the image
    
    Assumptions:
        - We're working with only the MIAS dataset
    
    Raises:
        ValueError: If the input is neither an int nor a str
    """
    if isinstance(img_info, str):
            file_path = img_info
    elif isinstance(img_info, int):
        file_number = str(img_info).zfill(3)
        file_name = "mdb" + file_number + ".pgm"
        file_path = DATASET_DIR + file_name
    else:
        raise ValueError('You must pass in either the image number or path.')

    # open the image
    img = Image.open(file_path)

    # show the image
    plt.imshow(img)
    plt.show()

In [9]:
def display_rand_images_grid(image_paths, rows=5, cols=5):
    """
    Displays images from a directory in a grid.
    
    Parameters:
        image_paths(str): Directory containing images
        rows(int): Number of rows in the grid. 5 by default.
        cols(int): Number of columns in the grid. 5 by default.
    """
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(15,15))

    for i in range(rows):
        for j in range(cols):
            rand_index = random.randrange(len(image_paths))
            ax[i, j].axis('off')
            ax[i, j].imshow(read_img_as_np_array(image_paths[rand_index], rgb=True))

In [10]:
# display random dataset images in a grid 
display_rand_images_grid(image_paths)

## Preprocessing

In [11]:
def left_align(img):
    """
    Determines whether the breast is aligned to the right or left side of the image
    by measuring the mean gray level of either half. Flips the image to the left if it
    is right-aligned.
    
    Parameters:
        img: The numpy array representing the image.
    
    Assumptions:
        - The half of the image on which the majority of the breast region lies has a higher mean than the othe half
        - The input image is LCC view
    
    Returns:
        numpy array with the left aligned image.
    """
    pixels = np.asarray(img)
    if np.mean(pixels[0:256, 0:128]) < np.mean(pixels[0:256, 128:256]):
        return pixels[:, ::-1]
    return pixels 

In [42]:
def perform_contrast(img):
    """
    Adjusts the contrast of the given image by a specific factor.
    
    Parameters:
        img: PIL image to be adjusted
    Assumptions:
        None
    Returns:
        PIL image with the contrast adjusted.
    """
    enhancer = ImageEnhance.Contrast(img)
    factor = 1#increase contrast
    img = enhancer.enhance(factor)
    return img 

In [43]:
def remove_bar(img, starting_pixel):
    """
    Crops the image to remove any top and left bar (white space).
    
    Parameters:
        img: numpy array of the image to be adjusted
        starting_pixel: a seed indicating the top-left x, y coordinates where the area of interest begins
    Assumptions:
        - The input image is LMLO view
    Returns:
        Numpy array of the cropped image
    """
    return img[starting_pixel[1]:, starting_pixel[0]:256]

In [44]:
def find_top_left_pixel(img):
    """
    Approximately finds the top left pixel where the mammogram begins.
    
    Parameters:
        img: The PIL image
    
    Assumptions:
        - There is no blank space at the top of the image (not even 1px)
        - We want to optimize for speed over precision
        - The input image is LMLO view
    
    Returns:
        int: The x, y position where the pectoral muscle approximately begins
    """
    # get all the pixel values
    px = img.load()
    
    # loop through the pixels and read the value
    # tip: using a step of 2 for the row and 5 for the column to speed up the iteration by 10x .. 
    # .. Remove the step factor is precision is more important.
    for r_px in range(0, img.size[1], 2):
        for c_px in range(0, img.size[0], 5):
            # checking the pixel value against a threshold
            if px[c_px, r_px] > 50:
                return c_px, r_px

def test_find_top_left_pixel():
    """
    Runs the find_top_left_pixel() method and visualizes the results.
    """
    # setting up the plot
    rows = 3
    cols = 3
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(15,15))

    for i in range(rows):
        for j in range(cols):
            # picking a random image (left-aligned)
            img_num = random.randrange(0, len(image_paths)+1, 2)
            file_number = str(img_num).zfill(3)
            file_name = "mdb" + file_number + ".pgm"
            file_path = DATASET_DIR + file_name
            
            # open the image
            img = Image.open(file_path)

            # get the x value of the top-left pixel
            c_pixel, r_pixel = find_top_left_pixel(img)

            # plot a big box to visualize the pixel location 
            for a in range(30):
                for b in range(30):
                    img.putpixel((c_pixel+a,r_pixel+b), 255)

            ax[i, j].axis("off")
            ax[i, j].set_title(str(file_number))
            ax[i, j].imshow(img)

In [45]:
# test_find_top_left_pixel()

In [46]:
def preprocess_image(img):
    """
    Combines all of the above preprocessing steps together on a given image.
    
    Parameters:
        img: PIL image to be adjusted
    Assumptions:
        - There is no blank space at the top of the image (not even 1px).
        - We want to optimize for speed over precision.
        - The input image is LCC view.
        - The half of the image on which the majority of the breast region lies has a higher mean than the othe half.
        
    Returns:
        PIL image ready for the level set algorithm.
    """
    img = img.resize((256,256))
    starting_pixel = find_top_left_pixel(img)
    img = left_align(img)
    img = remove_bar(img, starting_pixel)
    img = np.interp(img, [np.min(img), np.max(img)], [0, 255])
    return img

## Parameter Initialisation

In [47]:
def initialise_params(preprocessed_img):
    """
    Parametes:
        preprocessed_img: Input image that has been preprocessed. It will be passed as a parameter 
        to the level set algorithm
    
    Returns:
        a dictionary containing all the parameters needed for the algorithm.
    """
    # initialize LSF as binary step function
    c0 = 2
    initial_lsf = c0 * np.ones(preprocessed_img.shape)
    # generate the initial region R0 as two rectangles
    initial_lsf[0:10, 0:10] = -c0 # top left corner

    # parameters
    return {
        'img': preprocessed_img,
        'initial_lsf': initial_lsf,
        'timestep': 7,  # time step
        'iter_inner': 10,
        'iter_outer': 20,
        'lmda': 5,  # coefficient of the weighted length term L(phi)
        'alfa': -3,  # coefficient of the weighted area term A(phi)
        'epsilon': 1.2,  # parameter that specifies the width of the DiracDelta function
        'sigma': 0.8,  # scale parameter in Gaussian kernel
        'potential_function': DOUBLE_WELL,
    }

In [48]:
# use single well potential p1(s)=0.5*(s-1)^2, which is good for region-based model
DOUBLE_WELL = 'double-well'

# use double-well potential in Eq. (16), which is good for both edge and region based models
SINGLE_WELL = 'single-well'

## Function that runs the DLRSE algorithm {iter_outer} times

In [49]:
def find_lsf(img: np.ndarray, initial_lsf: np.ndarray, timestep=1, iter_inner=10, iter_outer=30, lmda=5,
             alfa=-3, epsilon=1.5, sigma=0.8, potential_function=DOUBLE_WELL):
    """
    Parameters:
        img: Input image as a grey scale uint8 array (0-255).
        initial_lsf: Array as same size as the img that contains the seed points for the LSF.
        timestep: Time Step.
        iter_inner: How many iterations to run drlse before showing the output.
        iter_outer: How many iterations to run the iter_inner.
        lmda: coefficient of the weighted length term L(phi).
        alfa: coefficient of the weighted area term A(phi).
        epsilon: parameter that specifies the width of the DiracDelta function.
        sigma: scale parameter in Gaussian kernal.
        potential_function: The potential function to use in drlse algorithm. Should be SINGLE_WELL or DOUBLE_WELL.
        
    Returns:
        Phi value that indicates the boundary drawn. Will be used to segment the image.
    """
    if len(img.shape) != 2:
        raise Exception("Input image should be a gray scale one")

    if len(img.shape) != len(initial_lsf.shape):
        raise Exception("Input image and the initial LSF should be in the same shape")

    if np.max(img) <= 1:
        raise Exception("Please make sure the image data is in the range [0, 255]")

    # parameters
    mu = 0.2 / timestep  # coefficient of the distance regularization term R(phi)

    img_smooth = np.array(img, dtype='float32')
#     img_smooth = gaussian_filter(img, sigma)  # smooth image by Gaussian convolution
    [Iy, Ix] = np.gradient(img_smooth)
    f = np.square(Ix) + np.square(Iy)
    g = 1 / (1 + f)  # edge indicator function.

    # initialize LSF as binary step function
    phi = initial_lsf.copy()

    show_fig1(phi)
    show_fig2(phi, img)
#     print('show fig 2 first time')

    if potential_function != SINGLE_WELL:
        potential_function = DOUBLE_WELL  # default choice of potential function

    # start level set evolution
    for n in range(iter_outer):
        phi = drlse_edge(phi, g, lmda, mu, alfa, epsilon, timestep, iter_inner, potential_function)
        print('show fig 2 for %i time' % n)
        draw_all(phi, img)

    # refine the zero level contour by further level set evolution with alfa=0
    alfa = 0
    iter_refine = 10
    phi = drlse_edge(phi, g, lmda, mu, alfa, epsilon, timestep, iter_refine, potential_function)
    return phi


## DLRSE Algorithm

In [50]:
def drlse_edge(phi_0, g, lmda, mu, alfa, epsilon, timestep, iters, potential_function):  # Updated Level Set Function
    """
    Parameters:
        phi_0: level set function to be updated by level set evolution.
        g: edge indicator function.
        lmda: weight of the weighted length term.
        mu: weight of distance regularization term.
        alfa: weight of the weighted area term.
        epsilon: width of Dirac Delta function.
        timestep: time step.
        iters: number of iterations.
        potential_function: choice of potential function in distance regularization term.
                            As mentioned in the above paper, two choices are provided: potentialFunction='single-well' or
                            potentialFunction='double-well', which correspond to the potential functions p1 (single-well)
                            and p2 (double-well), respectively.
    
    Returns:
        An updated level set function.
                
    """
    phi = phi_0.copy()
    [vy, vx] = np.gradient(g)
    for k in range(iters):
        phi = neumann_bound_cond(phi)
        [phi_y, phi_x] = np.gradient(phi)
        s = np.sqrt(np.square(phi_x) + np.square(phi_y))
        delta = 1e-10
        n_x = phi_x / (s + delta)  # add a small positive number to avoid division by zero
        n_y = phi_y / (s + delta)
        curvature = div(n_x, n_y)

        if potential_function == SINGLE_WELL:
            dist_reg_term = laplace(phi, mode='nearest') - curvature  # compute distance regularization term in equation (13) with the single-well potential p1.
        elif potential_function == DOUBLE_WELL:
            dist_reg_term = dist_reg_p2(phi)  # compute the distance regularization term in eqaution (13) with the double-well potential p2.
        else:
            raise Exception('Error: Wrong choice of potential function. Please input the string "single-well" or "double-well" in the drlse_edge function.')
        dirac_phi = dirac(phi, epsilon)
        area_term = dirac_phi * g  # balloon/pressure force
        edge_term = dirac_phi * (vx * n_x + vy * n_y) + dirac_phi * g * curvature
        phi += timestep * (mu * dist_reg_term + lmda * edge_term + alfa * area_term)
    return phi


def dist_reg_p2(phi):
    """
    Compute the distance regularization term with the DOUBLE WELL potential p2 in equation (16).
    
    Parameter:
        phi: level set function to be updated by level set evolution.
        
    Returns
        Distance regression term in equation (13).
    
    """
    [phi_y, phi_x] = np.gradient(phi)
    s = np.sqrt(np.square(phi_x) + np.square(phi_y))
    a = (s >= 0) & (s <= 1)
    b = (s > 1)
    ps = a * np.sin(2 * np.pi * s) / (2 * np.pi) + b * (s - 1)  # compute first order derivative of the double-well potential p2 in equation (16)
    dps = ((ps != 0) * ps + (ps == 0)) / ((s != 0) * s + (s == 0))  # compute d_p(s)=p'(s)/s in equation (10). As s-->0, we have d_p(s)-->1 according to equation (18)
    return div(dps * phi_x - phi_x, dps * phi_y - phi_y) + laplace(phi, mode='nearest')


def div(nx: np.ndarray, ny: np.ndarray) -> np.ndarray:
    """
    Compute the curvature to the level set function.
    Returns:
        The curvature of the level set function. 
    """
    [_, nxx] = np.gradient(nx)
    [nyy, _] = np.gradient(ny)
    return nxx + nyy


def dirac(x: np.ndarray, sigma: np.ndarray) -> np.ndarray:
    """
    Parameters:
        x: level set function to be updated by level set evolution.
        sigma: Width of Dirac Delta function.
        
    Returns:
        Updated function that will be used to calculate the area and the edge term.
    """
    f = (1 / 2 / sigma) * (1 + np.cos(np.pi * x / sigma))
    b = (x <= sigma) & (x >= -sigma)
    return f * b


def neumann_bound_cond(f):
    """
    Make a function satisfy Neumann boundary condition.
    
    Parameter: 
        f: level set function to be updated by level set evolution.
    Return:
        A level set function that satisfy neumann boundary condition.
    """
    g = f.copy()

    g[np.ix_([0, -1], [0, -1])] = g[np.ix_([2, -3], [2, -3])]
    g[np.ix_([0, -1]), 1:-1] = g[np.ix_([2, -3]), 1:-1]
    g[1:-1, np.ix_([0, -1])] = g[1:-1, np.ix_([2, -3])]
    return g

## Visualisation Code

In [51]:
plt.ion()
fig1 = plt.figure(1)
fig2 = plt.figure(2)


def show_fig1(phi: np.ndarray):
    """
    Visualise the changes of the level set function in 3 dimensionals.
    
    Parameter:
        phi: level set function obtained from the dlrse algorithm.
    """
    fig1.clf()
    ax1 = fig1.add_subplot(111, projection='3d')
    y, x = phi.shape
    x = np.arange(0, x, 1)
    y = np.arange(0, y, 1)
    X, Y = np.meshgrid(x, y)
    ax1.plot_surface(X, Y, -phi, rstride=2, cstride=2, color='r', linewidth=0, alpha=0.6, antialiased=True)
    ax1.contour(X, Y, phi, 0, colors='g', linewidths=2)


def show_fig2(phi: np.ndarray, img: np.ndarray):
    """
    Visualise the contour changes. 
    
    Parameters:
        phi: level set function obtained from the dlrse algorithm.
        img: the segmented image.
    """
    fig2.clf()
    contours = measure.find_contours(phi, 0)
    ax2 = fig2.add_subplot(111)
    ax2.imshow(img, interpolation='nearest', cmap=plt.get_cmap('gray'))
    for n, contour in enumerate(contours):
        ax2.plot(contour[:, 1], contour[:, 0], linewidth=2)


def draw_all(phi: np.ndarray, img: np.ndarray, pause=0.3):
    """
    Call show_fig1() and show_fig2()
    Parameters:
        phi: level set function obtained from the dlrse algorithm.
        img: the segmented image.
        pause: number of seconds delay before exiting the function.
    """
    show_fig2(phi, img)
    # uncomment the following line if you want to see the 3D gradient evolution
    show_fig1(phi)
    
    plt.pause(pause)

## Saving Output

In [52]:
def save_segmented_image(phi: np.ndarray, img: np.ndarray):
    """
    Assign the pixel within the boundary to be black. Save the output image. Return the segmented and original image for 
    visualisation purpose.
    
    Parameters:
        phi: finalised level set function (provide the boundary drawn).
        img: img to be segmented.
        filename: file name used to save the image.
    
    Returns:
        Tuple that contains the numpy arrays for the original image and the segmented image
    """
    original_img = img.copy()
    contours = measure.find_contours(phi, 0)
    max_of_y = int(np.round(max(contours[0], key=lambda x: x[0]))[0])
    # contains the rightmost pixel at that particular y 
    record_array = [0] * (max_of_y + 1)
    for contour in contours[0]:
        x, y = int(contour[1]), round(contour[0]) 
        if record_array[y] < x:
            record_array[y] = x
    for i in range(len(record_array)):
        for j in range(record_array[i]):
            img[i,j] = 0
    im = Image.fromarray(img)
    im = im.convert('L')
    im.save("output/segmented_" + file_name)
    return img, original_img

## Main Code

In [53]:
ori_and_modified_photo_tuple_top_20 = [] # store a list of tuple, each tuple contains the numpy array 
                                         # of the original image and the modified image

for i in range(21,41):
    file_number = str(i).zfill(3)
    file_name = "mdb" + file_number + ".pgm"
    file_path = DATASET_DIR + file_name
    img = Image.open(file_path)
    contrasted_img = perform_contrast(img)
    processed_img = preprocess_image(contrasted_img)
    params = initialise_params(processed_img)
    # params = two_cells_params()
    phi = find_lsf(**params)

    # print('Show final output')
    draw_all(phi, params['img']) # currently no pause

    processed_img_for_download = preprocess_image(img)
    img, original_img = save_segmented_image(phi, processed_img_for_download)
    ori_and_modified_photo_tuple_top_20.append((original_img, img))

show fig 2 for 0 time
show fig 2 for 1 time
show fig 2 for 2 time
show fig 2 for 3 time
show fig 2 for 4 time
show fig 2 for 5 time
show fig 2 for 6 time
show fig 2 for 7 time
show fig 2 for 8 time
show fig 2 for 9 time
show fig 2 for 10 time
show fig 2 for 11 time
show fig 2 for 12 time
show fig 2 for 13 time
show fig 2 for 14 time
show fig 2 for 15 time
show fig 2 for 16 time
show fig 2 for 17 time
show fig 2 for 18 time
show fig 2 for 19 time
show fig 2 for 0 time
show fig 2 for 1 time
show fig 2 for 2 time
show fig 2 for 3 time
show fig 2 for 4 time
show fig 2 for 5 time
show fig 2 for 6 time
show fig 2 for 7 time
show fig 2 for 8 time
show fig 2 for 9 time
show fig 2 for 10 time
show fig 2 for 11 time
show fig 2 for 12 time
show fig 2 for 13 time
show fig 2 for 14 time
show fig 2 for 15 time
show fig 2 for 16 time
show fig 2 for 17 time
show fig 2 for 18 time
show fig 2 for 19 time
show fig 2 for 0 time
show fig 2 for 1 time
show fig 2 for 2 time
show fig 2 for 3 time
show fig 2 f

KeyboardInterrupt: 

In [ ]:
counter = 0
x_axis = 0

fig, ax = plt.subplots(nrows=10,ncols=4, figsize=(12.5,12.5))
for original_img, img in ori_and_modified_photo_tuple_top_20:
    y_axis= math.floor(counter)
    ax[y_axis, x_axis].axis('off')
    ax[y_axis, x_axis].imshow(original_img)
    x_axis += 1
    ax[y_axis, x_axis].axis('off')
    ax[y_axis, x_axis].imshow(img)
    x_axis +=1 
    if x_axis > 3:
        x_axis = 0
    counter += 0.5